# imports

In [1]:

import pandas as pd
import numpy as np

import pandas_ta as ta



In [2]:
def pivot_point( df) :
  pivot_points = []


  for _, row in df.iterrows():
      high = row['High']
      low = row['Low']
      close = row['Close']

      pivot_point = (high + low + close) / 3


      pivot_points.append(round(pivot_point, 2))

  return pivot_points


# Assuming 'df' is your DataFrame with a 'price' column
def candle_color(df):
    color = []

    for idx in df.index:
        if df['Open'][idx] > df['Close'][idx]:
            color.append(-1)
        elif df['Open'][idx] < df['Close'][idx]:
            color.append(1)
        else:
            color.append(0)  

    return color





def fair_value_gap(df):
    high, low = df['High'], df['Low']
    fvg = [None] * len(df)
    for row in range(1, len(df) - 1):
        # bulish
        if not high[row - 1] >= low[row + 1]:
            # Placeholder for the missing logic
            fvg[row] = (low[row + 1] - high[row - 1])  # Example: Storing the row index in the fvg list
        # Bearish
        if not low[row - 1] <= high[row + 1]:
            # Placeholder for the missing logic
            fvg[row] = ( high[row + 1] - low[row - 1])  # Example: Storing the row index in the fvg list
        
    return fvg




# variables

In [3]:
#Reading Data
df = pd.read_csv("C:\\Users\AmirHussain\Desktop\machine-Learning-tradert-bot-main\Datas\XAUUSDM5.csv" , 
                   encoding = 'utf-16', 
                   names =  ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', '0'] )

df.drop(['0', 'Volume'], inplace= True, axis= 1)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', drop=True, inplace = True)


In [4]:
df['fvg'] = fair_value_gap(df)

np.argmax(df['fvg'])

12353

In [5]:
df.iloc[np.argmax(df['fvg'])]

Open     1920.90
High     1928.03
Low      1914.36
Close    1919.67
fvg        26.14
Name: 2022-02-28 01:00:00, dtype: float64

In [ ]:
from backtesting import Strategy

def SIGNAL():
    return df['signal']

class myStrategy(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        
    
    def next(self):
        super().next()
        
        if self.signal1 == 1:
            sl1 = self.data.Low[-1] - 5
            tp1 = self.data.High[-1] + 5
            self.buy(sl=sl1, tp=tp1)
        
        if self.signal1 == 2:
            sl1 = self.data.High[-1] + 5
            tp1 = self.data.Low[-1] - 5
            self.sell(sl=sl1, tp=tp1)

from backtesting import Backtest
m1 = pd.read_csv("C:\\Users\AmirHussain\Desktop\machine-Learning-tradert-bot-main\Datas\XAUUSDM1.csv" , 
                   encoding = 'utf-16', 
                   names =  ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', '0'] )
m1['Date'] = pd.to_datetime(df['Date'])
bt = Backtest(df, myStrategy,cash=10_000, commission= 0.001,hedging= False, margin= .01, trade_on_close = False, exclusive_orders=False)
bt.run()

In [ ]:
# bt.plot()

# Plot For testing

In [6]:
import plotly.offline as pyo
import plotly.graph_objects as go


a = np.argmax(df['fvg'])
c= np.argmax(df['fvg']) - 10
t = np.argmax(df['fvg']) +10
Open = df['Open'][c:t]
Close = df['Close'][c:t]
High = df['High'][c:t]
Low = df['Low'][c:t]
x = df.index[c:t]
# sma1 = df['ma 1'][14212:14216]
# sma2 = df['ma 2'][14212:14216]

# signal = df['signal'][14212:14216]
# color = df['color'][14212:14216]
# fvg = df['fvg'][14212:14216]

fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=x,
    open=Open,
    high=High,
    low=Low,
    close=Close,
    increasing=dict(line=dict(color='green')),
    decreasing=dict(line=dict(color='red')),
    name='Candlestick'
))

# for i in x:
#     if signal[i] == 1:
#         fig.add_trace(go.Scatter(x=[i], y=[Open[i]], mode='markers', marker=dict(color='black')))
#     elif signal[i] == 2:
#         fig.add_trace(go.Scatter(x=[i], y=[Open[i]], mode='markers', marker=dict(color='white')))

# fig.add_trace(go.Scatter(x=[0, 1], y=[a, a], mode='lines', name='Horizontal Line'))

# for i in x:
#     if fvg[i] >= 1.3:
#         fig.add_trace(go.Scatter(x=[i], y=[Open[i]], mode='markers', marker=dict(color='black')))

# fig.add_trace(go.Scatter(x=x, y=sma1, mode='lines', marker=dict(color='red')))
# fig.add_trace(go.Scatter(x=x, y=sma2, mode='lines', marker=dict(color='green')))

pyo.init_notebook_mode(connected=True)
pyo.iplot(fig)
print(df.iloc[1])

Open     1920.90
High     1928.03
Low      1914.36
Close    1919.67
fvg        26.14
Name: 2022-02-28 01:00:00, dtype: float64
